In [1]:
from src.static_variables import (HDF5_DATA_FILENAME, X_TRAIN_DIR, X_TEST_DIR, Y_TRAIN_DIR, Y_TEST_DIR, SPLIT_DATASETS_DIR)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import h5py
import pickle
from scipy.io import arff
from src.run_experiments import RunExperiments

In [2]:
from src.files_io import FilesIO
from src.models_container import ModelsContainer

In [3]:
#load dataset
abalone = arff.loadarff('Delgado_data/abalone/abalone.arff')
abalone = pd.DataFrame(abalone[0])
wine_quality = arff.loadarff('Delgado_data/wine-quality-red/wine-quality-red.arff')
wine_quality = pd.DataFrame(wine_quality[0])

In [4]:
#create new instance of FilesIO class
f = FilesIO('data/test.hdf5')

In [5]:
#save original dataset in HDF5 database
metadata = [
    {'class_name':'clase', 'dataset_name':'abalone'},
    {'class_name':'clase', 'dataset_name':'wine_quality'},       
           ]
f.save_datasets(datasets =[abalone, wine_quality], 
                dataset_names=['abalone','wine_quality'], 
                dts_metadata=metadata)

/media/viktor/Data/PhD/ml_experiments/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->bytes,key->block1_values] [items->['clase']]

  


In [6]:
#create train /test split and save to database
f.split_and_save(dataset_paths=['abalone','wine_quality'], save_loc='split_datasets')



/media/viktor/Data/PhD/ml_experiments/src/files_io.py:171: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->bytes,key->values] [items->None]

  verbose = None)


In [23]:
#alternative split without saving to database
X_train, X_test, y_train, y_test = f.split_dataset('abalone', test_size=0.33)
y_train = np.array(y_train, dtype=int).ravel()
y_test = np.array(y_test, dtype=int).ravel()

In [24]:
models_container = ModelsContainer()
experiments = RunExperiments()

In [25]:
#example to instantiate models
instantiated_models = models_container.instantiate_models(RandomForestClassifier=None, SVM=None)

In [26]:
modelling_strategies = models_container.create_models()
trained_models, _ = experiments.run_experiments( X_train=X_train, y_train=y_train, model_container=modelling_strategies)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:    3.9s finished
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    1.6s finished


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.0s finished
